## ML Baseline для прогноза скорости коррозии

В этом ноутбуке мы построим простой, но корректный базовый ML-пайплайн:
- загрузим данные;
- подготовим признаки (числовые, категориальные, материал);
- обучим несколько базовых моделей;
- корректно оценим качество с кросс-валидацией;
- интерпретируем важность признаков;
- сохраним результаты.

По ходу добавлены пояснения ключевых терминов и ссылки на источники.


In [6]:
# Импорты и версия окружения
import os, sys, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print('Python:', sys.version)
print('pandas:', pd.__version__)
print('numpy:', np.__version__)

# Добавляем src в путь, чтобы импортировать загрузку данных
if '../src' not in sys.path:
    sys.path.append('../src')
from database import load_corrosion_data

# Попытка импортировать scikit-learn
try:
    import sklearn
    from sklearn.model_selection import train_test_split, KFold, GroupKFold
    from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
    from sklearn.linear_model import Ridge
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.preprocessing import StandardScaler
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline
    print('scikit-learn:', sklearn.__version__)
except Exception as e:
    print('Внимание: требуется scikit-learn. Установите из requirements.txt. Ошибка:', e)


Python: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
pandas: 2.3.3
numpy: 2.3.4
scikit-learn: 1.7.2


### Что такое кросс-валидация (CV)?

Кросс-валидация — это метод оценки качества модели, при котором данные многократно делятся на обучающую и валидационную части, а итоговое качество усредняется. Это уменьшает риск «подгонки» к конкретному разбиению.

- Обзор: [Документация scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html)
- KFold: делит данные на K равных частей и по очереди использует одну часть для валидации.
- GroupKFold: аналог KFold, но соблюдает группировку (например, все записи одной установки идут либо в train, либо в val), чтобы избежать утечки информации между фолдами.


In [8]:
# Загрузка данных и базовая проверка

df = load_corrosion_data()
print(f"Записей: {len(df):,}; колонок: {len(df.columns)}")
print('Колонки:', list(df.columns)[:20], '...')

# Целевая переменная
TARGET = 'corrosion_rate'
if TARGET not in df.columns:
    raise ValueError(f'Нет целевой переменной {TARGET} в данных')

# Базовая фильтрация по физическим ограничениям
before = len(df)
df = df[df[TARGET].notna()]
df = df[(df[TARGET] >= 0) & (df[TARGET] <= 10)]
print(f"Удалено записей: {before - len(df):,}; осталось: {len(df):,}")

# Возможные групповые колонки для GroupKFold (выберем первую доступную)
possible_groups = ['installation','equipment','component_type_name']
group_col = next((c for c in possible_groups if c in df.columns), None)
print('Групповая колонка для CV:', group_col)


Записей: 442,052; колонок: 64
Колонки: ['id', 'installation', 'equipment', 'mms', 'measurement_date', 'measurement', 'is_replaced', 'corrosion_rate', 'component_type_id', 'component_type_name', 'wall_thickness', 'radius', 'outer_diameter', 'inner_diameter', 'diameter_to_thickness_ratio', 'cross_sectional_area', 'material_type', 'material_resistance_score', 'water_content', 'h2s_content'] ...
Удалено записей: 128,920; осталось: 313,132
Групповая колонка для CV: installation


### Подготовка признаков: что и зачем

- Числовые признаки: передаем в модель как есть, часто полезно масштабирование (StandardScaler) для линейных моделей.
- Категориальные признаки: преобразуем в числовой вид. В простом варианте — one-hot encoding (фиктивные переменные).
- Материал: `material_resistance_score` — числовой признак устойчивости материала.

Полезно:
- One-Hot Encoding: [Документация scikit-learn](https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features)
- Масштабирование: [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)


In [9]:
# Формирование списков признаков

# Базовые группы признаков (подстраиваемся под доступность колонок)
num_candidates = [
    'wall_thickness','outer_diameter','inner_diameter','radius',
    'diameter_to_thickness_ratio','cross_sectional_area',
    'operating_temperature','operating_pressure',
    'water_content','h2s_content','sulfur_content','chlorine_content',
    'co2_content','oxygen_content','methane_content','ethane_content',
    'propane_content','butane_content','propylene_content','ethylene_content',
    'total_sulfur_compounds','total_chlorine_compounds','total_acids',
    'material_resistance_score'
]

cat_candidates = ['component_type_name']

num_features = [c for c in num_candidates if c in df.columns]
cat_features = [c for c in cat_candidates if c in df.columns]

print('Числовые признаки:', num_features)
print('Категориальные признаки:', cat_features)

# Простая обработка категорий: ограничим OHE топ-10 наиболее частых значений
TOP_K = 10
cat_maps = {}
for c in cat_features:
    top_vals = df[c].value_counts().nlargest(TOP_K).index
    df[c] = df[c].where(df[c].isin(top_vals), other='Other')
    cat_maps[c] = {'top_values': list(top_vals), 'other':'Other'}

X_num = df[num_features].copy()
X_cat = df[cat_features].copy() if cat_features else pd.DataFrame(index=df.index)

y = df[TARGET].astype(float)


Числовые признаки: ['wall_thickness', 'outer_diameter', 'inner_diameter', 'radius', 'diameter_to_thickness_ratio', 'cross_sectional_area', 'operating_temperature', 'operating_pressure', 'water_content', 'h2s_content', 'sulfur_content', 'chlorine_content', 'co2_content', 'oxygen_content', 'methane_content', 'ethane_content', 'propane_content', 'butane_content', 'propylene_content', 'ethylene_content', 'total_sulfur_compounds', 'total_chlorine_compounds', 'total_acids', 'material_resistance_score']
Категориальные признаки: ['component_type_name']


In [10]:
# Построение матрицы признаков X

# One-Hot Encoding вручную через pandas.get_dummies
if len(cat_features) > 0:
    X_cat_encoded = pd.get_dummies(X_cat, columns=cat_features, drop_first=True, dtype=np.uint8)
else:
    X_cat_encoded = pd.DataFrame(index=df.index)

X = pd.concat([X_num, X_cat_encoded], axis=1)
print('Итоговая матрица X:', X.shape)
print('Пропуски в X:', X.isna().sum().sum())

# Простейшая импутация числовых пропусков медианами
for c in num_features:
    if c in X.columns:
        X[c] = X[c].fillna(X[c].median())

print('Пропуски после импутации:', X.isna().sum().sum())


Итоговая матрица X: (313132, 30)
Пропуски в X: 2732219
Пропуски после импутации: 0


### Метрики качества: что измеряем

- MAE (Mean Absolute Error) — средняя абсолютная ошибка, интерпретируется в единицах цели. [MAE](https://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-error)
- RMSE (Root Mean Squared Error) — корень из средней квадратичной ошибки; сильнее штрафует большие ошибки. [MSE/RMSE](https://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-error)
- R² — доля объясненной вариации; 1 — идеально, 0 — не лучше константы. [R2](https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score)


In [11]:
# Бейзлайн на простом train/val сплите

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Линейная модель с регуляризацией (Ridge)
ridge = Pipeline(steps=[
    ('scaler', StandardScaler(with_mean=False)),  # with_mean=False для разреженных OHE
    ('model', Ridge(alpha=1.0, random_state=42))
])

ridge.fit(X_train, y_train)
ridge_pred = ridge.predict(X_val)

mae = mean_absolute_error(y_val, ridge_pred)
rmse = np.sqrt(mean_squared_error(y_val, ridge_pred))
r2 = r2_score(y_val, ridge_pred)

print(f'Ridge — MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}')

# Деревья — RandomForest
rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=None,
    n_jobs=-1,
    random_state=42
)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)

mae_rf = mean_absolute_error(y_val, rf_pred)
rmse_rf = np.sqrt(mean_squared_error(y_val, rf_pred))
r2_rf = r2_score(y_val, rf_pred)

print(f'RandomForest — MAE: {mae_rf:.4f}, RMSE: {rmse_rf:.4f}, R2: {r2_rf:.4f}')


Ridge — MAE: 0.0630, RMSE: 0.1798, R2: 0.0182
RandomForest — MAE: 0.0606, RMSE: 0.1715, R2: 0.1071


In [13]:
# Кросс-валидация (CV): KFold или GroupKFold

K = 5

cv_results = []

if group_col is not None:
    splitter = GroupKFold(n_splits=K)
    groups = df[group_col]
    cv_iter = splitter.split(X, y, groups=groups)
    cv_name = f'GroupKFold({group_col})'
else:
    splitter = KFold(n_splits=K, shuffle=True, random_state=42)
    cv_iter = splitter.split(X, y)
    cv_name = 'KFold'

print('CV схема:', cv_name)

for fold, (tr_idx, va_idx) in enumerate(cv_iter, 1):
    X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    # Ridge
    ridge = Pipeline(steps=[('scaler', StandardScaler(with_mean=False)), ('model', Ridge(alpha=1.0, random_state=42))])
    ridge.fit(X_tr, y_tr)
    pred = ridge.predict(X_va)
    cv_results.append({'model':'Ridge','fold':fold,
                       'MAE': mean_absolute_error(y_va, pred),
                       'RMSE': np.sqrt(mean_squared_error(y_va, pred)),
                       'R2': r2_score(y_va, pred)})

    # RandomForest
    rf = RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=42)
    rf.fit(X_tr, y_tr)
    pred_rf = rf.predict(X_va)
    cv_results.append({'model':'RandomForest','fold':fold,
                       'MAE': mean_absolute_error(y_va, pred_rf),
                       'RMSE': np.sqrt(mean_squared_error(y_va, pred_rf)),
                       'R2': r2_score(y_va, pred_rf)})

cv_df = pd.DataFrame(cv_results)
print('Средние метрики по фолдам:')
print(cv_df.groupby('model')[['MAE','RMSE','R2']].agg(['mean','std']).round(4))


CV схема: GroupKFold(installation)
Средние метрики по фолдам:
                 MAE            RMSE              R2        
                mean     std    mean     std    mean     std
model                                                       
RandomForest  0.0723  0.0310  0.1646  0.1222 -0.2273  0.1669
Ridge         0.0694  0.0271  0.1578  0.1248 -0.0729  0.0756


In [14]:
# Интерпретация: важности признаков и коэффициенты

# Дообучим RandomForest на всех данных (для демонстрации важностей)
rf_full = RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=42)
rf_full.fit(X, y)

importances = pd.Series(rf_full.feature_importances_, index=X.columns).sort_values(ascending=False)
print('Топ-15 важнейших признаков (RF):')
print(importances.head(15))

# Коэффициенты Ridge (на всех данных) — перед этим подгоним scaler
ridge_full = Pipeline(steps=[('scaler', StandardScaler(with_mean=False)), ('model', Ridge(alpha=1.0, random_state=42))])
ridge_full.fit(X, y)

# Достаем коэффициенты из последнего шага
coef = pd.Series(ridge_full.named_steps['model'].coef_, index=X.columns).sort_values(key=np.abs, ascending=False)
print('\nТоп-15 по |коэффициентам| (Ridge):')
print(coef.head(15))


Топ-15 важнейших признаков (RF):
operating_temperature          0.168457
butane_content                 0.096846
operating_pressure             0.091162
total_sulfur_compounds         0.084687
sulfur_content                 0.075416
water_content                  0.072870
wall_thickness                 0.067589
cross_sectional_area           0.062219
diameter_to_thickness_ratio    0.056793
inner_diameter                 0.040283
component_type_name_Участок    0.029080
component_type_name_Отвод      0.023891
h2s_content                    0.020911
outer_diameter                 0.019547
radius                         0.019137
dtype: float64

Топ-15 по |коэффициентам| (Ridge):
cross_sectional_area          -0.027966
total_sulfur_compounds         0.025317
diameter_to_thickness_ratio   -0.018749
h2s_content                   -0.015037
wall_thickness                 0.014059
outer_diameter                 0.013748
radius                         0.013748
component_type_name_Отвод      0.013

In [17]:
# Сохранение артефактов

os.makedirs('../data', exist_ok=True)

# Преобразуем сводку CV к плоским строковым ключам
summary = cv_df.groupby('model')[['MAE','RMSE','R2']].agg(['mean','std']).round(6)
summary.columns = [f'{m}_{s}' for m, s in summary.columns]
cv_summary = summary.reset_index().to_dict(orient='records')

# Гарантируем сериализацию значений в JSON (приводим к float/str)
rf_top = {str(k): float(v) for k, v in importances.head(20).round(6).items()}
ridge_top = {str(k): float(v) for k, v in coef.head(20).round(6).items()}

results = {
    'cv_summary': cv_summary,
    'top_rf_features': rf_top,
    'top_ridge_coeffs_abs': ridge_top,
    'group_column': group_col
}

with open('../data/ml_baseline_results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print('Сохранено: ../data/ml_baseline_results.json')


Сохранено: ../data/ml_baseline_results.json
